<a href="https://colab.research.google.com/github/Toroi0610/worksoave/blob/master/RBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import altair as alt

# 機械学習を用いて量子多体系を表現する
量子多体系とは多数の自由度がお互いに
相互作用しあう系を指し，そこにおける各
自由度の運動は多体の波動関数によって支
配される．多体波動関数さえわかってしま
えば問題解決であるが，多体系のハミルト
ニアンの次元は自由度の数に対して指数関
数的に増大するため，自由度の数が増える
と多体波動関数を厳密に求めることは不可
能になる．そのため，多体波動関数をいか
に精度よく表現できるか？という問題は，
非常に重要な課題になる．
この問題に対して新しい風が吹いている．
これまで物理的洞察に基づいて波動関数を
近似する試みは多くなされているが，それ
とは逆に対称性などの明らかな拘束条件以
外には直観に頼らず，機械学習の力を借り
て波動関数を構築しようという動きである．
機械学習は膨大なデータセットからその本
質的なパターンを抽出する．すなわち，本
来の波動関数は指数関数的に大きな次元を
持つベクトルとみなせるが，その本質的な
パターンを機械学習によって見つけ出し，
波動関数の次元よりもはるかに少ない数の
パラメータを用いて波動関数を精度よく近
似しようという試みである．
機械学習の手法の中でも，本稿では，人
工ニューラル・ネットワークの一種である
ボルツマンマシンを用いた変分波動関数に
ついて議論する．ボルツマンマシンは可視
（入力）層の自由度である可視ユニットに
加え，仮想的自由度である隠れ層の不可視
ユニットで構成され，ユニット間が結合
（相互作用）を持つ構造をしている．ボル
ツマンマシンは可視ユニットの状態配置を
入力とし，その生成確率分布を学習する機
械である．それに対し，多体波動関数は物
理的なハミルトニアンの自由度の状態配置
それぞれに対して値を与える．波動関数の
値が，それらの状態配置に対する（複素数
に一般化された）確率であると考えると，
物理的ハミルトニアンの自由度と可視ユ
ニット自由度を同一視することで，多体波
動関数をボルツマンマシンによって書き下
すことができる．すなわちボルツマンマシ
ンをユニット間結合定数などを変分パラ
メータとする変分波動関数とみなし，物理
的ハミルトニアンのエネルギー期待値を最
小化するように波動関数を最適化し（機械
学習の言語ではこれが学習に対応），未知
の基底状態を探索する．
最初に量子多体系に適用されたのは，最
も単純な構造をしている制限ボルツマンマ
シン（Restricted Boltzmann machine，略して
RBM）である．RBM を用いた変分波動関
数は量子スピン系に適用され，その精度の
良さが実証された．
この研究を皮切りに RBM 波動関数の特
性が以下のようにわかってきた．i）不可視
ユニットの数を指数関数的に増やすとどん
な波動関数も表現可能．ii）相互作用が短
（長）距離だと，エンタングルメント・エ
ントロピーが表面（体積）則を満たすこと．
iii）従来の波動関数法を組み合わせると
フェルミオン系などにも適応可能となりよ
り良い精度が達成できること．iv）隠れ層
を一層増やした深層ボルツマンマシン
（DBM）にすると関数表現能力が格段に向
上する．その性質を用いると，DBM を用
いて基底状態の波動関数を厳密に構築でき
ること．
ボルツマンマシン波動関数の研究は始
まったばかりである．その有用性，より深
い学理が近い将来明らかになるだろう．

# 1.はじめに
近年，物理の分野でも機械学習の有用性の吟味が始まっ
ている．本稿では，その手法の中でも人工ニューラル・
ネットワークを物性物理に適用した例を取り上げる．まず
どのような適用例があるのかをここで簡単に紹介する．
機械学習は大量のデータからデータの分布に関する何ら
かのモデルを構築する（＝本質的なパターンを抽出する）．
モデルはその目的によって，識別モデルと生成モデルに大
別される．識別モデルではあるデータが特定のクラスに所
属する確率を直接モデル化し，そのモデルに基づいてクラ
スの境界を学習するものである．犬と猫の画像データセッ
トに対して，犬・猫である確率をモデル化して，犬か猫か
に分類するようなことを考えていただければ良い．このよ
うに識別モデルでは与えられた観測データがどのような生
成確率に基づいているかは考慮しない．逆に生成モデルで
は，それぞれのクラスにおけるデータ分布がどのような確
率分布で生成されるかをモデル化する．
識別モデルの物理への適用例としては量子相の分類，相
境界決定・相転移検出 1‒10）が代表的である．例えば系の波
動関数の実空間分布を画像だと思って，知られた画像解析
手法を使い，量子相を分類する．
生成モデルの適用例としては，量子多体系のソル
バー 11‒23）や，モンテカルロ法の効率化 24, 25）などが挙げら
れる．前者では，人工ニューラル・ネットワークを用いて，
多体系の波動関数を表す．後者では，通常のメトロポリス
法などに則って生成したモンテカルロサンプルを学習機械
に入力し，背景にある生成確率分布を学習させ，効率的な
クラスター更新を提案する試みなどがある．
本稿では，生成モデルに注目し，量子多体系の波動関数
を表現し，多体系のソルバーとして用いる研究を紹介する．
様々な人工ニューラル・ネットワークの中でもボルツマン
マシンを使った波動関数表現に関して解説する．まず次節
でボルツマンマシンについて紹介する．3 節では RBM を
使った波動関数表現の手法を導入し，4 節において，その
性質や数値結果について議論する．5 節と 6 節では，ボル
ツマンマシン波動関数を改良するための試みを紹介する．

# 2. ボルツマンマシン
波動関数表現への応用を議論する前に，人工ニューラル・ネットワークの一種であるボルツマンマシン，26） 特にRBM 27）と深層ボルツマンマシン（DBM: deep Boltzmann
machine）の構造と表現能力についてまとめておこう．
## 2.1制限ボルツマンマシン(RBM)
図 1（a）に RBM の構造を示す．RBM は，人工ニューロンの自由度である N 個の可視ユニットと M 個の不可視ユニットから構成される．それぞれの自由度が存在する領域を可視層，不可視（隠れ）層と呼ぶ．可視，不可視ユニットの状態を表す変数をそれぞれ $v_i,  h_j (i＝1, ..., N, j＝1, ..., M)$とし，各ユニットは 2 つの状態を取り得るとする．その状態値は $v_i , h_j＝0, 1$ で定義されることが多いが，ここでは
RBM 波動関数を導入した論文 11）に従って，それと等価な$v_i , h_j＝±1$（イジング変数）で定義する．RBM では一般のボルツマンマシン（図 1（b））と異なり，可視ユニット間同士や不可視ユニット間同士には相互作用がなく，層間の結合しかないという制約がある．そのため，制限ボルツマンマシン（Restricted Boltzmann machine）と呼ばれている．RBM は可視ユニットのイジングスピン変数の任意の配置が生じる確率を近似する生成モデルである．下の議論で明らかになるが，不可視ユニットがあることによって確率を
近似する際の柔軟性が生まれている．可視・不可視ユニットの状態の配置をそれぞれ $v＝(v_1 , ..., v_N)，h＝(h_1 , ..., h_M)$とし，それぞれの状態の配置に応じて仮想的なエネルギー関数$ E(v, h)$を

$E(v,h)=-\sum_{i}a_iv_i-\sum_jb_jh_j-\sum_{i,j}{W}_{ij}v_ih_j$

と定義する．ここで $a_i , b_j$ はバイアスと呼ばれる．それぞれのユニットの状態をイジングスピン自由度だと考え，式（1）を仮想的なイジング模型のエネルギ－と考えると，これらの項は磁場に対応する．$W_{ij}$ は隠れ層と可視層の間の結合（スピン間の相互作用）である．分配関数$Z＝\sum_{v,h}e^{-E(v, h)}$を用いて，ボルツマン重み（統計力学的確率分布関数）$p(v,h)$を $p(v,h)=e^{-E(v,h)}/Z$と定義する．$v, h$ に関する和は
イジング自由度に対する状態和（トレース）をとる（トレースアウトする）ことを意味する．この分布はボルツマン分布に他ならないので，任意の確率分布を表現するのには能力が足りない．しかし，不可視ユニットの状態配置 h に対する部分状態和をとった $v$ に対する確率分布関数（$v$ のみを変数とする確率を与えるので，周辺分布とよばれる）

$\tilde{p}(v)=\sum_hp(v,h) (\sum_v\tilde{p}(v)=1)$

を考えると，$\tilde{p}(v)$は変分パラメータ $a_i , b_j , W_{ij}$ をうまく選ぶことにより v に対する任意の確率分布を精度よく近似できる．数学的には$v$が表す $2^N$個の配置に対する確率分布を近似する際，指数関数的に大きな数$(M～2^N)$の不可視ユニットを用意すれば厳密に表現できる 28, 29）ことが知られている．しかし，指数関数的に大きな数の不可視ユニットは現実的な計算では扱えないため，実際の計算では$N$ の
冪関数で増える数の不可視ユニットを用いる．このように RBM は一般のボルツマンマシン同様，多変数関数を近似する「機械」であるが，不可視ユニット間の相互作用がないために，不可視ユニットに関して一体問題に帰着し,$h$に対する部分状態和を解析的に行えることがRBM の特長である．*1 実際，周辺分布 $\tilde{p}(v)$に対して，

$\tilde{p}(v)=e^{\sum_ia_iv_i}\times\prod_j2\cosh{(b_j+\sum_iW_{ij}v_i)}$

という解析的な表式を簡単に得ることができ，数値計算の際のコストを大幅に削減できる．